In [17]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Generate State file with Vaccination 

In [2]:
df_state = pd.read_csv("/home/swiadmin/test/csv/latest/states.csv")

df_state["Date"] = pd.to_datetime(df_state["Date"])

df_state_vacc = pd.read_csv("/home/swiadmin/test/csv/latest/cowin_vaccine_data_statewise.csv")

cols = ['Updated On','State','Total Doses Administered',
        'First Dose Administered','Second Dose Administered']

df_state_vacc = df_state_vacc[cols]

df_state_vacc['Updated On'] = df_state_vacc['Updated On'].apply(lambda _: datetime.strptime(_,"%d/%m/%Y"))

df_state_vacc = df_state_vacc.merge(df_state,how="right",left_on=["Updated On","State"],right_on=["Date","State"])

df_state_vacc["Vaccination3"] = df_state_vacc["Total Doses Administered"] - df_state_vacc["First Dose Administered"] - df_state_vacc["Second Dose Administered"]

df_state_vacc = df_state_vacc.drop(['Updated On','Total Doses Administered'],axis=1)

df_state_vacc = df_state_vacc.rename(columns = {"First Dose Administered":"Vaccination1","Second Dose Administered":"Vaccination2"})

df_state_vacc.to_csv("states_vacc.csv")

# Generate District file with vaccination

In [13]:
df_districts = pd.read_csv("/home/swiadmin/test/csv/latest/districts.csv")

In [29]:
df_districts.columns

Index(['Date', 'State', 'District', 'Confirmed', 'Recovered', 'Deceased',
       'Other', 'Tested'],
      dtype='object')

In [21]:
df_district_vacc = pd.read_csv("/home/swiadmin/test/csv/latest/cowin_vaccine_data_districtwise.csv",header=[0,1])

In [30]:
df_district_vacc.columns

MultiIndex([(        'S No',                            ' '),
            (  'State_Code',                            ' '),
            (       'State',                            ' '),
            ('District_Key',                            ' '),
            (   'Cowin Key',                            ' '),
            (    'District',                            ' '),
            (  '16/01/2021', 'Total Individuals Registered'),
            (  '16/01/2021',                     'Sessions'),
            (  '16/01/2021',                       'Sites '),
            (  '16/01/2021',      'First Dose Administered'),
            ...
            (  '21/02/2022',     'Total Doses Administered'),
            (  '22/02/2022',      'First Dose Administered'),
            (  '22/02/2022',     'Second Dose Administered'),
            (  '22/02/2022',     'Total Doses Administered'),
            (  '23/02/2022',      'First Dose Administered'),
            (  '23/02/2022',     'Second Dose Administ

In [60]:
df_temp = df_district_vacc["24/02/2022"].merge(df_district_vacc["District"],left_index=True,right_index=True)
# df_temp = df_temp.rename(columns={" ","District"})
df_district_vacc["District"].stack(-1)

0                         Nicobars
1         North and Middle Andaman
2                    South Andaman
3                        Anantapur
4                         Chittoor
                    ...           
749                        Purulia
750                        Birbhum
751              South 24 Parganas
752                 Uttar Dinajpur
753              Paschim Bardhaman
Length: 754, dtype: object

In [64]:
df_temp["Date"] = [24/02/2022
for idx in df_temp.index:
    

Nicobars
North and Middle Andaman
South Andaman
Anantapur
Chittoor
East Godavari
Guntur
Krishna
Kurnool
Prakasam
S.P.S. Nellore
Srikakulam
Visakhapatnam
Vizianagaram
West Godavari
Y.S.R. Kadapa
Anjaw
Changlang
Upper Dibang Valley
East Kameng
East Siang
Papum Pare
Kamle
Kra Daadi
Kurung Kumey
Lepa Rada
Lohit
Longding
Lower Dibang Valley
Lower Siang
Lower Subansiri
Namsai
Pakke Kessang
Papum Pare
Shi Yomi
Siang
Tawang
Tirap
Upper Siang
Upper Subansiri
West Kameng
West Siang
Baksa
Barpeta
Biswanath
Bongaigaon
Cachar
Charaideo
Chirang
Darrang
Dhemaji
Dhubri
Dibrugarh
Dima Hasao
Goalpara
Golaghat
Hailakandi
Hojai
Jorhat
Kamrup Metropolitan
Kamrup
Karbi Anglong
Karimganj
Kokrajhar
Lakhimpur
Majuli
Morigaon
Nagaon
Nalbari
Sivasagar
Sonitpur
South Salmara Mankachar
Tinsukia
Udalguri
West Karbi Anglong
Araria
Arwal
Aurangabad
Banka
Begusarai
Bhagalpur
Bhojpur
Buxar
Darbhanga
East Champaran
Gaya
Gopalganj
Jamui
Jehanabad
Kaimur
Katihar
Khagaria
Kishanganj
Lakhisarai
Madhepura
Madhubani
Munger
Mu